In [1]:
from datasets import load_dataset

# dataset = load_dataset("csv", data_files="Data/user_defined_utterances.csv")
dataset = load_dataset("csv", data_files={"train": ["Data/user_train.csv"], "valid": "Data/user_test.csv"})

/Users/samehstudy/sourcecode/nlpmvp/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-b7cf248e1f7aa5f0
Reusing dataset csv (/Users/samehstudy/.cache/huggingface/datasets/csv/default-b7cf248e1f7aa5f0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 2/2 [00:00<00:00, 819.04it/s]


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
model_ckpt = 'nickprock/xlm-roberta-base-banking77-classification'
# model_ckpt = 'miguelvictor/python-gpt2-large'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

Some weights of the model checkpoint at nickprock/xlm-roberta-base-banking77-classification were not used when initializing XLMRobertaModel: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at nickprock/xlm-roberta-base-banking77-classification and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

In [3]:
def mean_pooling(model_output, attention_mask):
    # Extract the token embeddings
    token_embeddings = model_output[0]
    # Compute the attention mask
    input_mask_expanded = (attention_mask
        .unsqueeze(-1)
        .expand(token_embeddings.size())
        .float())
    # Sum the embeddings, but ignore masked tokens
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    # Return the average as a single vector
    return sum_embeddings / sum_mask

In [4]:
def embed_text(examples):
    inputs = tokenizer(examples["text"], padding=True, truncation=True,
                        max_length=128, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    pooled_embeds = mean_pooling(model_output, inputs["attention_mask"])
    return {"embedding": pooled_embeds.cpu().numpy()}

In [5]:
tokenizer.pad_token = tokenizer.eos_token
embs_train = dataset["train"].map(embed_text, batched=True, batch_size=16)
embs_valid = dataset["valid"].map(embed_text, batched=True, batch_size=16)

100%|██████████| 2/2 [00:00<00:00,  2.73ba/s]


In [6]:
embs_train.add_faiss_index("embedding")

100%|██████████| 1/1 [00:00<00:00, 880.23it/s]


Dataset({
    features: ['text', 'category', 'embedding'],
    num_rows: 79
})

In [7]:
embs_valid

Dataset({
    features: ['text', 'category', 'embedding'],
    num_rows: 20
})

In [10]:
import numpy as np

i, k = 0, 3 # Select the first query and 3 nearest neighbors
rn, nl = "\r\n\r\n", "\n" # Used to remove newlines in text for compact display

query =  np.array(embs_valid[i]["embedding"], dtype=np.float32)
scores, samples = embs_train.get_nearest_examples("embedding", query, k=k)

# print(f"QUERY LABELS: {embs_valid[i]['category']}")
# print(f"QUERY TEXT:\n{embs_valid[i]['text'][:200].replace(rn, nl)} [...]\n")
# print("="*50)
# print(f"Retrieved documents:")
# for score, label, text in zip(scores, samples["category"], samples["text"]):
#     print("="*50)
#     print(f"TEXT:\n{text[:200].replace(rn, nl)} [...]")
#     print(f"SCORE: {score:.2f}")
#     print(f"LABELS: {label}")

: 

: 

In [ ]:
import numpy as np

def get_sample_preds(sample, m):
    return (np.sum(sample["label_ids"], axis=0) >= m).astype(int)

In [ ]:
# def find_best_k_m(ds_train, valid_queries, valid_labels, max_k=17):
#     max_k = min(len(ds_train), max_k)
#     perf_micro = np.zeros((max_k, max_k))
#     perf_macro = np.zeros((max_k, max_k))
#     for k in range(1, max_k):
#         for m in range(1, k + 1):
#             samples = ds_train.get_nearest_examples_batch("embedding",
#             valid_queries,
#             k=k)
#             y_pred = np.array([get_sample_preds(s, m) for s in samples])
#             clf_report = classification_report(valid_labels, y_pred,
#             target_names=mlb.classes_, zero_division=0, output_dict=True)
#         perf_micro[k, m] = clf_report["micro avg"]["f1-score"]
#         perf_macro[k, m] = clf_report["macro avg"]["f1-score"]
# return perf_micro, perf_macro